In [156]:
pip install pandas kagglehub sentence-transformers rake-nltk spacy PyPDF2 pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
168376.90s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 23.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 9.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Research

## Load data

In [138]:
import pandas as pd
import kagglehub
import os

resumes_df_path = "data/UpdatedResumeDataSet.csv"
jobs_df_path = "data/data job posts.csv"

if not os.path.exists(resumes_df_path):
    print(f"Resumes dataset not found at {resumes_df_path}")
    print(f"Downloading from kaggle...")
    path = kagglehub.dataset_download("gauravduttakiit/resume-dataset")
    !mkdir -p data
    !mv $path/* ./data

if not os.path.exists(jobs_df_path):
    print(f"Jobs dataset not found at {jobs_df_path}")
    print(f"Downloading from kaggle...")
    path = kagglehub.dataset_download("madhab/jobposts")
    !mkdir -p data
    !mv $path/* ./data


resumes_df = pd.read_csv(resumes_df_path)
jobs_df = pd.read_csv(jobs_df_path)

In [139]:
resumes_df.describe()

,Category,Resume
count,962,962
unique,25,166
top,Java Developer,"Technical Skills Web Technologies: Angular JS,..."
freq,84,18


In [140]:
resumes_df.loc[100]["Category"]

'Advocate'

In [141]:
jobs_df = pd.read_csv("data/data job posts.csv")

In [142]:
jobs_df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [143]:
sample_5 = jobs_df.sample(5)["jobpost"]
for i in sample_5:
    print(i)
    print("\n\n")

"ACBA-Credit Agricole Bank" CJSC
TITLE:  Risk Management Specialist
OPEN TO/ ELIGIBILITY CRITERIA:  All qualified candidates
START DATE/ TIME:  ASAP
DURATION:  Long-term with 3 months probation period.
LOCATION:  Yerevan, Armenia
JOB DESCRIPTION:  ACBA-Credit Agricole Bank CJSC is seeking a Risk
Management Specialist to be responsible for monitoring and assessing
bank risks, analyzing the external financial markets, making forecasts,
preparing risk management reports.
JOB RESPONSIBILITIES:
- Monitor and assess Financial and Operational Risks;
- Prepare monthly, quarterly and annual risk management reports;
- Analyze the external financial market developments;
- Make forecasts;
- Follow the recommendations of the Head of Division.
REQUIRED QUALIFICATIONS:
- University degree in Economics;
- Advanced knowledge of financial markets, financial analysis,
statistics, banking legislation and normative field;
- Excellent knowledge of Armenian, Russian and English languages
(knowledge of Russia

Observation:
All jobpost are devided into section with capitalized titles, like TITLE, AGE, JOB DESCRIPTION etc...

If some of those section are more or less consistant across the dataset, we can peform some structuring

## Let's perform named entity recognition (NER) on resumes, to extract important information.

In [144]:
!python -m spacy download en_core_web_lg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
168218.90s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [145]:
import spacy
import pandas as pd
from collections import defaultdict

def extract_resume_entities(df, text_column='Resume'):
    """
    Extract relevant entities from resume text using spaCy.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing resumes
    text_column (str): Name of the column containing resume text
    
    Returns:
    pandas.DataFrame: DataFrame with original data and extracted entities
    """
    # Load English language model with parser and NER components
    nlp = spacy.load("en_core_web_lg")
    
    # Custom patterns for skills and education-related terms
    # TODO: move patterns to a separate file and add way more of them
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    patterns = [
        {"label": "SKILL", "pattern": [{"LOWER": {"IN": ["python", "java", "javascript", "sql", "r", "C", "C++",  "tensorflow",
                                                        "pytorch", "machine learning", "deep learning", "nlp",
                                                        "data analysis", "statistical analysis"]}}]},
        {"label": "DEGREE", "pattern": [{"LOWER": {"IN": ["phd", "master's", "bachelor's", "mba", "ms", "bs", "ba"]}}]}
    ]
    ruler.add_patterns(patterns)
    
    def process_text(text):
        if pd.isna(text):
            return {
                'organizations': [],
                'locations': [],
                'dates': [],
                'people': [],
                'skills': [],
                'degrees': [],
                'languages': [],
                'skill_phrases': []
            }
            
        # Process text through spaCy pipeline
        doc = nlp(text)
        
        # Initialize containers for different entity types
        entities = defaultdict(list)
        
        # Extract standard named entities
        for ent in doc.ents:
            if ent.label_ in ['ORG', 'GPE', 'DATE', 'PERSON']:
                entities[ent.label_].append(ent.text)
            
        # Extract custom entities (skills and degrees)
        for ent in doc.ents:
            if ent.label_ in ['SKILL', 'DEGREE']:
                entities[ent.label_].append(ent.text)
        
        # Extract noun chunks as potential skill phrases
        skill_phrases = [chunk.text for chunk in doc.noun_chunks 
                        if any(tech in chunk.text.lower() 
                              for tech in ["python", "java", "analysis", "engineering", "development", "computer"])]
        
        return {
            'organizations': list(set(entities['ORG'])),
            'locations': list(set(entities['GPE'])),
            'dates': list(set(entities['DATE'])),
            'people': list(set(entities['PERSON'])),
            'skills': list(set(entities['SKILL'])),
            'degrees': list(set(entities['DEGREE'])),
            'languages': list(set(entities['LAUGUAGE'])),
            'skill_phrases': list(set(skill_phrases))
        }
    
    # Create a list to store results
    results = []
    
    # Process each resume
    for _, row in df.iterrows():
        results.append(process_text(row[text_column]))
    
    # Convert results to DataFrame
    entities_df = pd.DataFrame(results)
    
    # Combine with original DataFrame
    result_df = pd.concat([df.reset_index(drop=True), entities_df], axis=1)
    
    return result_df

In [146]:
resumes_sample_df = resumes_df.sample(5)
enriched_df = extract_resume_entities(resumes_sample_df)

In [147]:
resumes_sample_df

,Category,Resume
818,ETL Developer,Computer skills: - Yes. SQL knowledge-yes Unix...
843,DotNet Developer,"Technical Skills CATEGORY SKILLS Language C, C..."
579,Python Developer,Training attended: 1. Successfully completed E...
671,Network Security Engineer,Communication Skills My writing skills in Engl...
759,Hadoop,Areas of expertise â¢ Big Data Ecosystems: Ha...


In [148]:
enriched_df

,Category,Resume,organizations,locations,dates,people,skills,degrees,languages,skill_phrases
0,ETL Developer,Computer skills: - Yes. SQL knowledge-yes Unix...,"[Informatica, Engineering Navi Mumbai, Playing...","[India, Mumbai]","[the past 3years, January 2008, January 2009 t...",[],[SQL],[],[],"[Computer skills, Engineering Navi Mumbai, Eng..."
1,DotNet Developer,"Technical Skills CATEGORY SKILLS Language C, C...","[CSS, Dashboard, Coding, Front-end Designing l...","[Madhya Pradesh, Ajax, Pune]","[today, 2013, 2012, 2007, 1 year months, all d...","[Marks, Gallery, RM Comments, Biography]","[JavaScript, SQL]",[MS],[],"[Computer Science Indore, CSS Browser Scriptin..."
2,Python Developer,Training attended: 1. Successfully completed E...,"[Platinum Auto, ESD, Computer Society of India...","[US, Pune, JSCOE]","[2018, 2015, 2016, June 2014, June 2012, June ...","[Git, Avishkar]",[Python],[],[],"[Engineering, Computer Society, computer visio..."
3,Network Security Engineer,Communication Skills My writing skills in Engl...,"[Sify, Tikona & HCL Comnet, TATA, Firewall, Ma...",[Jaunpur],"[June 2008, April 2010, April 2008, July 2012 ...",[],[],[],[],[]
4,Hadoop,Areas of expertise â¢ Big Data Ecosystems: Ha...,"[Hive, Oozie, Ruckus Wireless\r\nDescription, ...","[Chennai, Autosys, Pyspark, Anaconda]","[September 2016 to Till date, Aug 2015 to sep ...","[Tectia, Cassandra, Tez, Oozie, Putty, Respone...","[Java, Python]",[],[],"[Transaction monitoring system development, Ha..."


We can extract some entities like language, skills, education etc with acceptable accuracy. But for that to work we need better custom patterns.

### Let's now explore entities and ways to extract them in jobposts

In [149]:
jobs_df_sample = jobs_df.sample(5)
enriched_jobs_df = extract_resume_entities(jobs_df_sample, text_column='jobpost')
enriched_jobs_df

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Month,IT,organizations,locations,dates,people,skills,degrees,languages,skill_phrases
0,AECOM International Development\r\nTITLE: Dep...,"Aug 20, 2010",Deputy Chief of Party,AECOM International Development,NaN,NaN,NaN,NaN,Fall/ Winter 2010,5 years,...,8,False,"[Party, AECOM International Development Inc., ...","[Yerevan, Armenia, Fall/]","[Winter 2010, 5 years, 05 September 2010, 21 A...",[],[],[],[],"[AECOM International Development Inc., the dev..."
1,"Synergy International Systems, Inc. - Armenia\...","Sep 18, 2006",Java Developer,"Synergy International Systems, Inc. - Armenia",NaN,NaN,NaN,NaN,Immediately,Long term,...,9,True,"[Synergy International Systems, JSP, Career Ce...","[Yerevan, Armenia, U.S.]","[01 October 2006, At least 5 years, At least 3...",[Anahit Sargsyan],"[Java, SQL]",[MS],[],"[the\r\nsoftware development process, Computer..."
2,American Bar Association / Central European an...,"May 23, 2004",Receptionist,American Bar Association / Central European an...,NaN,NaN,NaN,NaN,"2nd or 3rd week of June, 2004 (three month\r\n...",NaN,...,5,False,"[the Senior Legal Advisor, Submit CV, Career C...","[Yerevan, Armenia, Tbilisi]","[the day, 2nd, 31 May 2004, daily, three month...",[],[],[],[],"[- Computer literacy - word processing, the co..."
3,Cargomatrix Inc. Armenian Branch\r\nTITLE: C#...,"Nov 11, 2014",C#/ .NET Developer/ Intern,Cargomatrix Inc. Armenian Branch,NaN,Full time,Students and graduates,NaN,15 December 2014,From 6 to 12 months,...,11,True,"[DBMS, OOD, Application Development Team, jQue...","[Yerevan, Armenia, New York, USA]","[day, 6 to 12 months, 1999, 11 December 2014, ...",[Developer/ Intern],"[JavaScript, SQL]",[MS],[],"[Application Development Team, a real life pro..."
4,Tower International Consultants CJSC\r\n\r\n\r...,"Jun 8, 2015",Accounting and Tax Assistant,Tower International Consultants CJSC,NaN,NaN,NaN,NaN,NaN,Long term with 3 months of probation period.,...,6,False,"[Accounting and Tax, Career Center, Accounting...","[Yerevan, Armenia]","[08 June 2015, 3 months, 22 June 2015]",[],[],[],[],[]


## Testing proposed approach

### 1. Preprocess text.

In [150]:
import re
import unicodedata

class TextPreprocessor:
    def __init__(self):
       pass


    def normalize(self, text: str) -> str:
        """
        Normalize text by performing the following operations:
        - Convert to lowercase
        - Remove extra whitespace
        - Handle unicode characters
        - Remove multiple spaces
        - Remove email addresses (optional)
        - Remove URLs (optional)
        - Remove phone numbers (optional)
        
        Args:
            text (str): Input text to normalize
            
        Returns:
            str: Normalized text
        """
        # Convert to lowercase
        text = text.lower()
        
        # Handle unicode characters
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        
        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', '', text)
        
        # Remove email addresses
        text = re.sub(r'\S+@\S+', '', text)
        
        # Remove phone numbers
        text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '', text)
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        return text


In [151]:
import spacy
import pandas as pd
from collections import defaultdict
import json


class EntityExtractor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_lg")
        self.ruler = self.nlp.add_pipe("entity_ruler", before="ner")
        self.entity_labels = []
        self.phrases = {}


    def load_patterns(self, patters_file: str = None):
        with open(patters_file) as f:
            patters_json = json.load(f)
            
        self.patterns = [
            {"label": entity_label, "pattern": [{"LOWER": {"IN": entities}}]} for entity_label, entities in patters_json.items()    
        ]
        self.entity_labels = list(patters_json.keys())

        self.ruler.add_patterns(self.patterns)

    def load_phrases(self, phrases_file: str = None):
        with open(phrases_file) as f:
            self.phrases = json.load(f)


    def extract(self, text):
        if pd.isna(text):
            return {
                'organizations': [],
                'locations': [],
                'dates': [],
                'people': [],
                'skills': [],
                'degrees': [],
                'languages': [],
                'skill_phrases': []
            }
            
        doc = self.nlp(text)
        
        entities = defaultdict(list)
        
        # Standard named entities
        for ent in doc.ents:
            if ent.label_ in ['LAUGUAGE']:
                entities[ent.label_].append(ent.text)
        
        # Custom entities
        for ent in doc.ents:
            if ent.label_ in self.entity_labels:
                entities[ent.label_].append(ent.text)
        
        # Extract noun chunks as potential skill phrases
        doc_phrases = {}

        for phrase_label, phrase_list in self.phrases.items():
            doc_phrases[phrase_label] = [chunk.text for chunk in doc.noun_chunks 
                            if any(tech in chunk.text.lower() 
                                for tech in phrase_list)]

        return dict(entities) | doc_phrases
        

In [152]:
entity_extractor = EntityExtractor()
entity_extractor.load_patterns("data/patterns.json")
entity_extractor.load_phrases("data/phrases.json")

text_preprocessor = TextPreprocessor()
# text = resumes_sample_df["Resume"].iloc[0]
text = jobs_df_sample["jobpost"].iloc[0]
text = text_preprocessor.normalize(text)

print(f"[INPUT TEXT]\n {text}\n\n") 
entities = entity_extractor.extract(text)
entities

[INPUT TEXT]
 aecom international development title: deputy chief of party start date/ time: fall/ winter 2010 duration: 5 years location: yerevan, armenia job description: aecom is seeking qualified candidates for a 5 year usaid-funded, economic growth program in armenia. the objective of the program is to raise incomes and employment in armenia by promoting growth in selected commodity chains or services with export potential. focusing on the role of small and medium-sized enterprises (smes), this program is expected to facilitate the development of competitive enterprises and value chains by stimulating innovation, enhancing workforce skills, accelerating new enterprise formation, improving access to finance, and addressing shortcomings in the business environment. required qualifications: - masters degree from an accredited university; - should be a senior development professional with at least 7-10 year experience working on usaid programs in armenia; - demonstrated knowledge of u

{'SKILL': ['go'],
 'SKILL-PHRASES': ['international development title',
  'the development',
  'finance',
  'the business environment',
  'a senior development',
  'usaid finance and administration regulations',
  'armenian and english languages',
  'application procedures',
  'your application letter',
  '21 august 2010 application deadline',
  'aecom international development inc',
  'a global economic and infrastructure development firm']}

In [153]:
resumes_sample_df = resumes_df.sample(1)

In [154]:
augmented_resume_df = extract_resume_entities(resumes_sample_df)

## PDF recognition

In [158]:
import PyPDF2
import pdfplumber

def extract_with_pypdf2(pdf_path):
    """
    Extract text from PDF using PyPDF2.
    Better for simple PDFs with basic text.
    """
    text = ""
    with open(pdf_path, 'rb') as file:
        # Create PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Get number of pages
        num_pages = len(pdf_reader.pages)
        
        # Extract text from each page
        for page_num in range(num_pages):
            # Get page object
            page = pdf_reader.pages[page_num]
            
            # Extract text from page
            text += page.extract_text() + "\n"
    
    return text

def extract_with_pdfplumber(pdf_path):
    """
    Extract text from PDF using pdfplumber.
    Better for complex PDFs with tables and formatted text.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        # Extract text from each page
        for page in pdf.pages:
            text += page.extract_text() + "\n"
            
    return text

def extract_with_tables(pdf_path):
    """
    Extract both text and tables from PDF using pdfplumber.
    Returns a tuple of (text, tables).
    """
    text = ""
    all_tables = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract text
            text += page.extract_text() + "\n"
            
            # Extract tables
            tables = page.extract_tables()
            if tables:
                all_tables.extend(tables)
    
    return text, all_tables

In [159]:
resume_text = extract_with_pdfplumber("data/Resume.pdf")

normalized_text = text_preprocessor.normalize(resume_text)
entities = entity_extractor.extract(normalized_text)
entities

{'DEGREE': ['undergraduate', 'bachelor'],
 'SKILL': ['c', 'c++', 'python', 'vulkan', 'opengl'],
 'SKILL-PHRASES': ['roman naumenko research engineer',
  'a computer vision engineer',
  'gpu-accelerated computer graphics',
  'systems engineering',
  'real-time applications',
  'embedded hardware',
  'software engineering',
  'computer science',
  'leadership',
  'convolutional neural network inference',
  'embedded platform',
  'a visual-inertial odom- etry application',
  'software-hardware integration',
  'optimization',
  'specific hardware',
  'developed tools',
  'the rendering system',
  'trainee software developer nov 2021 - dec 2021 amc bridge (remote) dnipro',
  'research department',
  'contributedtoaplatformresearchprojectbyadaptingacnn-basedalikesolutionfor keypoint detection',
  'computers',
  'small mamba language model',
  'tookpartinresearchofacompletelydifferentapproachforbuildingfundamentalmodels']}

In [160]:
pypdf2_text = extract_with_pypdf2("data/Resume.pdf")
normalized_text = text_preprocessor.normalize(pypdf2_text)
entities = entity_extractor.extract(normalized_text)
entities

{'DEGREE': ['undergraduate', 'bachelor'],
 'SKILL': ['c', 'c++', 'python', 'vulkan', 'opengl'],
 'SKILL-PHRASES': ['roman naumenko research engineer',
  'a computer vision engineer',
  'gpu-accelerated computer graphics',
  'systems engineering',
  'real-time applications',
  'embedded hardware',
  'software engineering',
  'computer science',
  'leadership',
  'convolutional neural network inference',
  'embedded platform',
  'a visual-inertial odom- etry application',
  'software-hardware integration',
  'optimization',
  'specific hardware',
  'developed tools',
  'the rendering system',
  'trainee software developer nov 2021 - dec 2021 amc bridge (remote) dnipro',
  'research department',
  'a platform research project',
  'a cnn-based alike solution',
  'computers',
  'small mamba language model',
  'research',
  'a completely different approach']}

## Skills embeddings

In [161]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Tuple


class EmbeddingModel:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
    
    def encode(self, text: str) -> np.ndarray:
        return self.model.encode(text)
    
    def encode_batch(self, texts: List[str]) -> np.ndarray:
        return self.model.encode(texts)

In [162]:
entity_extractor = EntityExtractor()
entity_extractor.load_patterns("data/patterns.json")
entity_extractor.load_phrases("data/phrases.json")

text_preprocessor = TextPreprocessor()
# text = resumes_sample_df["Resume"].iloc[0]
text = jobs_df_sample["jobpost"].iloc[0]
text = text_preprocessor.normalize(text)

entities = entity_extractor.extract(text)

In [163]:
print(entities)

{'SKILL': ['go'], 'SKILL-PHRASES': ['international development title', 'the development', 'finance', 'the business environment', 'a senior development', 'usaid finance and administration regulations', 'armenian and english languages', 'application procedures', 'your application letter', '21 august 2010 application deadline', 'aecom international development inc', 'a global economic and infrastructure development firm']}


In [164]:
embeddings = {}
embedding_model = EmbeddingModel()

for entity_label, entity_list in entities.items():
    if entity_list:
        entity_embeddings = embedding_model.encode_batch(entity_list)
        embeddings[entity_label] = entity_embeddings

for entity_name, embedds in embeddings.items():
    print(f"[{entity_name}]\n {embedds.shape}")

[SKILL]
 (1, 384)
[SKILL-PHRASES]
 (12, 384)


## Finally Matching

In [165]:
from sklearn.metrics.pairwise import cosine_similarity

class ResumeScorer:
    def __init__(self, entity_extractor: EntityExtractor, embedding_model: EmbeddingModel):
        self.embedding_model = embedding_model
        self.entity_extractor = entity_extractor

    def score(self, job_text: str, resume_text: str) -> float:
        job_entities = self.entity_extractor.extract(job_text)
        resume_entities = self.entity_extractor.extract(resume_text)

        joint_entity_labels = list(set(job_entities.keys()) & set(resume_entities.keys()))

        scores = {}
        for entity_label in joint_entity_labels:
            if job_entities[entity_label] and resume_entities[entity_label]:
                job_embedds = self.embedding_model.encode_batch(job_entities[entity_label])
                resume_embedds = self.embedding_model.encode_batch(resume_entities[entity_label])
                scores[entity_label] = cosine_similarity(job_embedds, resume_embedds).mean()
            else:
                scores[entity_label] = 0.0

        return scores

    def score_batch(self, job_text: str, resume_texts: List[str]) -> np.ndarray:
        pass
        

In [166]:
entity_extractor = EntityExtractor()
entity_extractor.load_patterns("data/patterns.json")
entity_extractor.load_phrases("data/phrases.json")

text_preprocessor = TextPreprocessor()
job_text = resumes_sample_df["Resume"].iloc[0]
resume_text = jobs_df_sample["jobpost"].iloc[0]

job_text = text_preprocessor.normalize(job_text)
resume_text = text_preprocessor.normalize(resume_text)

embedding_model = EmbeddingModel()
scorer = ResumeScorer(entity_extractor, embedding_model)

scores = scorer.score(job_text, resume_text)

# Development

In [167]:
import pandas as pd

from src.text_procesing import PDFReader, TextPreprocessor
from src.scoring import EntityExtractor, EmbeddingModel, ResumeScorer

In [168]:
resumes_df_path = "data/UpdatedResumeDataSet.csv"
jobs_df_path = "data/data job posts.csv"
resumes_df = pd.read_csv(resumes_df_path)
jobs_df = pd.read_csv(jobs_df_path)

Sample several jobs from dataframe to match them against my resume.

In [169]:
jobs_sample_5_df = jobs_df.sample(15)
jobs_sample_5_df = jobs_sample_5_df.loc[:, ["Title", "jobpost"]]
jobs_sample_5_df

,Title,jobpost
8417,Cashier,Cascade Insurance ICJSC\r\nTITLE: Cashier\r\n...
8919,English Language Courses,Career Center NGO\r\nTITLE: English Language ...
10539,Operations Manager,Aldo\r\nTITLE: Operations Manager\r\nSTART DA...
915,Trade Agents,Convect\r\nTITLE: Trade Agents\r\nSTART DATE/...
748,Office Manager,Avarayr Tour Company\r\nTITLE: Office Manager...
7153,Communication Coordinator,Armenian Datacom Company CJSC\r\nTITLE: Commu...
9411,Specialist on Distance Banking,"VTB Bank, Armenia CJSC\r\nTITLE: Specialist o..."
13856,Software Architect/ Team Lead,LTX-Credence Armenia LLC\r\nTITLE: Software A...
6864,Junior Specialist at PR & Marketing Department,Ameria CJSC\r\nTITLE: Junior Specialist at PR...
1477,Supervisor,Kraft Foods Caucasus\r\nTITLE: Supervisor\r\n...


Let's normalize jobposts

In [170]:
text_preprocessor = TextPreprocessor()

jobs_sample_5_df['jobpost'] = jobs_sample_5_df['jobpost'].apply(text_preprocessor.normalize)
jobs_sample_5_df

,Title,jobpost
8417,Cashier,cascade insurance icjsc title: cashier duratio...
8919,English Language Courses,career center ngo title: english language cour...
10539,Operations Manager,aldo title: operations manager start date/ tim...
915,Trade Agents,convect title: trade agents start date/ time: ...
748,Office Manager,avarayr tour company title: office manager sta...
7153,Communication Coordinator,armenian datacom company cjsc title: communica...
9411,Specialist on Distance Banking,"vtb bank, armenia cjsc title: specialist on di..."
13856,Software Architect/ Team Lead,ltx-credence armenia llc title: software archi...
6864,Junior Specialist at PR & Marketing Department,ameria cjsc title: junior specialist at pr & m...
1477,Supervisor,kraft foods caucasus title: supervisor locatio...


Now load, recognise and preprocess resume

In [171]:
pdf_reader = PDFReader()
resume_text = pdf_reader.read("data/Resume.pdf")
resume_text = text_preprocessor.normalize(resume_text)
resume_text

'roman naumenko research engineer +(380) 63-036-5711 lviv, ukraine gmail linkedin github about myself im a computer vision engineer with extensive experience in gpu-accelerated computer graphics. i have strong expertise in systems engineering, integration, and machine learning. my current work focuses on optimizing ai algorithms for real-time applications on embedded hardware. education undergraduate of software engineering, national technical university of ukraine, kyiv 2020-2022 bachelor of computer science, ukrainian catholic university, lviv expected 2025 skills technical skills c/c++, python, pytorch, numpy, pandas, linux, directx11/vulkan/opengl, glsl/hlsl soft skills attention to detail, teamwork, leadership, problem-solving languages ukrainian (native), english (upper-intermediate) other computer vision, machine learning, multithreading, operating systems, modern rendering techinques, compute shaders, bvhs, oop, ood experience research engineer may 2024 - now foxfour & ucu mach

Match resume and job

In [172]:
entity_extractor = EntityExtractor()
entity_extractor.load_patterns("data/patterns.json")
entity_extractor.load_phrases("data/phrases.json")

embedding_model = EmbeddingModel()
scorer = ResumeScorer(entity_extractor, embedding_model)

In [173]:
for _, row in jobs_sample_5_df.iterrows():
    job_text = row["jobpost"]
    scores = scorer.score(job_text, resume_text)
    print(f"[JOB TITLE] {row['Title']}")
    print(f"[SCORES] {scores}")
    print("\n\n")

[JOB TITLE] Cashier
[SCORES] {'SKILL': np.float32(0.15498789), 'SKILL-PHRASES': np.float32(0.14526801)}



[JOB TITLE] English Language Courses
[SCORES] {'SKILL': np.float32(0.15498789), 'SKILL-PHRASES': np.float32(0.12865163)}



[JOB TITLE] Operations Manager
[SCORES] {'SKILL': np.float32(0.17582014), 'SKILL-PHRASES': np.float32(0.16444775)}



[JOB TITLE] Trade Agents
[SCORES] {'SKILL': np.float32(0.29547375), 'SKILL-PHRASES': np.float32(0.16431063)}



[JOB TITLE] Office Manager
[SCORES] {'SKILL': np.float32(0.15498789), 'DEGREE': np.float32(0.3967615), 'SKILL-PHRASES': np.float32(0.16317102)}



[JOB TITLE] Communication Coordinator
[SCORES] {'SKILL': np.float32(0.15498789), 'DEGREE': np.float32(0.23892389), 'SKILL-PHRASES': np.float32(0.1668916)}



[JOB TITLE] Specialist on Distance Banking
[SCORES] {'SKILL': np.float32(0.15498789), 'SKILL-PHRASES': np.float32(0.18020566)}



[JOB TITLE] Software Architect/ Team Lead
[SCORES] {'SKILL': np.float32(0.34173414), 'DEGREE': np.float3

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
import numpy as np

class ResumeScorerDev:
    def __init__(self, entity_extractor: EntityExtractor, embedding_model: EmbeddingModel):
        self.embedding_model = embedding_model
        self.entity_extractor = entity_extractor

    def score(self, job_text: str, resume_text: str, similarity_method: str = "ratio",  thr: float = 0.5):        
        similarity_func = self._get_similarity_func(similarity_method, thr)

        job_entities = self.entity_extractor.extract(job_text)
        resume_entities = self.entity_extractor.extract(resume_text)
        joint_entity_labels = list(set(job_entities.keys()) & set(resume_entities.keys()))

        scores = {}
        for entity_label in joint_entity_labels:
            if job_entities[entity_label] and resume_entities[entity_label]:
                job_embedds = self.embedding_model.encode_batch(job_entities[entity_label])
                resume_embedds = self.embedding_model.encode_batch(resume_entities[entity_label])

                scores[entity_label] = similarity_func(job_embedds, resume_embedds)
            else:
                scores[entity_label] = 0.0

        return scores

    def score_batch(self, job_text: str, resume_texts: List[str], similarity_method: str = "ratio",  thr: float = 0.5):
        similarity_func = self._get_similarity_func(similarity_method, thr)

        job_entities = self.entity_extractor.extract(job_text)
        resumes_entities = [self.entity_extractor.extract(resume_text) for resume_text in resume_texts]

        # prepare job embeddings
        job_embedds = {}
        for entity_label, entity_list in job_entities.items():
            if entity_list:
                job_embedds[entity_label] = self.embedding_model.encode_batch(entity_list)

        # calculate scores for all resumes
        scores = []
        for resume_entities in resumes_entities:
            resume_scores = {}
            for entity_label, resume_entity_list in resume_entities.items():
                if entity_label in job_entities.keys() and resume_entity_list:
                    resume_embedds = self.embedding_model.encode_batch(resume_entity_list)
                    resume_scores[entity_label] = similarity_func(job_embedds[entity_label], resume_embedds)
                else:
                    resume_scores[entity_label] = 0.0
            scores.append(resume_scores)

        return scores




    def _get_similarity_func(self, similarity_method: str, thr: float):
        if similarity_method == "ratio":
            if thr is None:
                raise ValueError("Threshold must be provided for ratio similarity")
            return lambda x, y: self._ratio_similarity(x, y, thr)
        elif similarity_method == "mean":
            return self._mean_similarity
        elif similarity_method == "sum":
            return self._sum_similarity
        else:
            raise ValueError(f"Unknown similarity method: {similarity_method}")
        


    def _ratio_similarity(self, job_embedds: np.ndarray, resume_embedds: np.ndarray, thr: float = 0.5) -> float:
        _scores = cosine_similarity(job_embedds, resume_embedds)
        max_scores = _scores.max(axis=1)
        ratio = (max_scores > thr).sum() / job_embedds.shape[0]
        return ratio
    
    def _mean_similarity(self, job_embedds: np.ndarray, resume_embedds: np.ndarray) -> float:
        _scores = cosine_similarity(job_embedds, resume_embedds)
        max_scores = _scores.max(axis=1)
        return max_scores.mean()
    
    def _sum_similarity(self, job_embedds: np.ndarray, resume_embedds: np.ndarray) -> float:
        _scores = cosine_similarity(job_embedds, resume_embedds)
        max_scores = _scores.max(axis=1)
        return max_scores.sum()

In [175]:
scorer_dev = ResumeScorerDev(entity_extractor, embedding_model)

### Muliple jobs to single resume

In [176]:
for _, row in jobs_sample_5_df.iterrows():
    job_text = row["jobpost"]
    scores = scorer_dev.score(job_text, resume_text, "mean", thr=0.5)
    print(f"[JOB TITLE] {row['Title']}")
    print(f"[SCORES] {scores}")
    print("\n\n")

[JOB TITLE] Cashier
[SCORES] {'SKILL': np.float32(0.287019), 'SKILL-PHRASES': np.float32(0.32757947)}



[JOB TITLE] English Language Courses
[SCORES] {'SKILL': np.float32(0.287019), 'SKILL-PHRASES': np.float32(0.3225222)}



[JOB TITLE] Operations Manager
[SCORES] {'SKILL': np.float32(0.27958798), 'SKILL-PHRASES': np.float32(0.38108698)}



[JOB TITLE] Trade Agents
[SCORES] {'SKILL': np.float32(0.6435095), 'SKILL-PHRASES': np.float32(0.39595264)}



[JOB TITLE] Office Manager
[SCORES] {'SKILL': np.float32(0.287019), 'DEGREE': np.float32(0.41509214), 'SKILL-PHRASES': np.float32(0.37723732)}



[JOB TITLE] Communication Coordinator
[SCORES] {'SKILL': np.float32(0.287019), 'DEGREE': np.float32(0.2429846), 'SKILL-PHRASES': np.float32(0.3612424)}



[JOB TITLE] Specialist on Distance Banking
[SCORES] {'SKILL': np.float32(0.287019), 'SKILL-PHRASES': np.float32(0.3997249)}



[JOB TITLE] Software Architect/ Team Lead
[SCORES] {'SKILL': np.float32(0.6435095), 'DEGREE': np.float32(0.2429846), 

### Single job to multiple resumes

Let's sample a single job description

In [177]:
single_job_sample = jobs_df.sample(1)
job_text = single_job_sample["jobpost"].iloc[0]
job_text = text_preprocessor.normalize(job_text)
job_title = single_job_sample["Title"].iloc[0]

print(f"[JOB TITLE] {job_title}")
print(f"[JOB TEXT] {job_text}")

[JOB TITLE] Security Coordinator
[JOB TEXT] "finca" uco cjsc title: security coordinator term: full time start date/ time: asap duration: long term location: yerevan, armenia job description: n/a job responsibilities: - draft contracts on maintenance of security services; - control full and properly implementation of security related contracts; - provide security procedures documentation and their archiving; - instruct and facilitate security personnel; - analyze and provide the necessary information to relevant departments; - coordinate procedures of reacting to any emergency situation. required qualifications: - university degree in law or a related field; - at least 2 years of experience in security department, preferable in an international company; - knowledge of security system and policies; - excellent team player; - excellent communication and negotiation skills; - ability to train different target groups; - computer literacy (ms office); - excellent knowledge of armenian langu

And some resumes

In [178]:
sample_size = 15
resumes_sample_df = resumes_df.sample(sample_size)
resumes_sample_df["Resume"] = resumes_sample_df["Resume"].apply(text_preprocessor.normalize)
resumes_sample_df

,Category,Resume
452,SAP Developer,education details may 2013 master computer app...
743,Hadoop,"skill set: hadoop, map reduce, hdfs, hive, sqo..."
47,HR,key skills: a computerized accounting with tal...
443,SAP Developer,skills: a etl a data warehousing a sql/pl sql ...
41,HR,"i.t. skills a windows xp, ms office (word, exc..."
838,DotNet Developer,"technical skills a web technologies: asp .net,..."
501,Electrical Engineering,skills: 1) mc office 2) autocad 2016 3) introd...
453,SAP Developer,education details january 2016 bachelor of eng...
383,Java Developer,"operating systems windows xp, 7, 10. tools/pac..."
243,Sales,it skills: ms office. photoshop. sql server.ed...


Nice, we've got a sales lad here.

In [179]:
resumes_text_list = resumes_sample_df["Resume"].tolist()
resumes_title_list = resumes_sample_df["Category"].tolist()

In [180]:
scores = scorer_dev.score_batch(job_text, resumes_text_list, "mean")

In [181]:
for i, score in enumerate(scores):
    print(f"[RESUME TITLE] {resumes_title_list[i]}")
    print(f"[SCORES] {score['SKILL-PHRASES']}")
    print("\n\n")

[RESUME TITLE] SAP Developer
[SCORES] 0.25588154792785645



[RESUME TITLE] Hadoop
[SCORES] 0.2737942039966583



[RESUME TITLE] HR
[SCORES] 0.3186565041542053



[RESUME TITLE] SAP Developer
[SCORES] 0.380808562040329



[RESUME TITLE] HR
[SCORES] 0.4100002348423004



[RESUME TITLE] DotNet Developer
[SCORES] 0.32414552569389343



[RESUME TITLE] Electrical Engineering
[SCORES] 0.3720112442970276



[RESUME TITLE] SAP Developer
[SCORES] 0.37141796946525574



[RESUME TITLE] Java Developer
[SCORES] 0.43605563044548035



[RESUME TITLE] Sales
[SCORES] 0.27039802074432373



[RESUME TITLE] DevOps Engineer
[SCORES] 0.5299204587936401



[RESUME TITLE] Database
[SCORES] 0.3475596010684967



[RESUME TITLE] Data Science
[SCORES] 0.365432471036911



[RESUME TITLE] PMO
[SCORES] 0.3619470000267029



[RESUME TITLE] Python Developer
[SCORES] 0.327668696641922



